In [11]:
%matplotlib widget
%matplotlib inline
import sys
from sys import platform
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pickle
import seaborn as sns
import scipy
import emcee
import corner
from IPython.display import display, Math
from tqdm import tqdm
from multiprocessing import Pool

from astropy.io import fits, ascii
from astropy.table import Table
from astropy.modeling import functional_models, fitting

import stingray.events as ev
import stingray.lightcurve as lc
from stingray import io
import stingray.powerspectrum as powspec 
import stingray.crossspectrum as crossspec
from hendrics.efsearch import dyn_folding_search, z_n_search, folding_search
import stingray.gti as sting_gti
import stingray.pulse.pulsar as plsr
from stingray import stats


sns.set_context('talk')
# sns.set_style("whitegrid")
sns.set_palette("colorblind")

sys.path.insert(1, '/Users/sean/scripts/helpers')

from stingray_plus import *

def minimize_remainder(arr, min_div, max_div):
    divisors = np.linspace(min_div, max_div, num=100)
    remainders = []
    for div in divisors:
        remainders.append(np.sum(np.mod(arr, div)))
        
    return divisors[np.argmin(remainders)]

def power_law(f, B, gamma):
    return B*np.power(f,gamma)

def Lorentzian(f, peakf, Q, A):
    gamma = peakf/(2.0 * Q)
    return (A * np.square(gamma)/(np.pi*gamma*(np.square(f-peakf) + np.square(gamma))))

def Lorentzian_C(f, peakf, Q, A, C):
    return Lorentzian(f, peakf, Q, A) + C

def Lorentzian_power(f, peakf, Q, A, B, gamma):
    return Lorentzian(f, peakf, Q, A) + power_law(f, B, gamma)

def N_Lorentzians(f, *args):
    N = int((len(args)-1)/3)
    peaks = args[:N]
    Qs = args[N:N+N]
    As = args[N+N:N+N+N]
    B, gamma = args[-2:]
    model = power_law(f, B, gamma)
    for i in range(N):
        gamma = peaks[i]/(2.0 * Qs[i])
        model = model + (As[i] * np.square(gamma)/(np.pi*gamma*(np.square(f-peaks[i]) + np.square(gamma))))
        
    return model


def QPO_scan(cross_spec, f_min=1e-4, f_max=2000., f_bin=1000):
    f_mask = (cross_spec.freq > f_min) * (cross_spec.freq < f_max)
    freq_steps = np.logspace(np.log10(cross_spec.freq[f_mask][0]), np.log10(cross_spec.freq[f_mask][-1]), f_bin + 2)
    xdata = cross_spec.freq[f_mask]
    ydata = cross_spec.power[f_mask]
    sigma = cross_spec.power_err[f_mask]
    
    pl_popt, pl_pcov = scipy.optimize.curve_fit(power_law, xdata, ydata, sigma = sigma, p0 = [10., -1.0], \
                                                bounds=np.array([(0.0, np.inf), (-np.inf, 0.0)]).T)
    print(pl_popt)
    chisq0 = np.sum(((ydata - power_law(xdata, *pl_popt)) / sigma) ** 2)
    chisq = []
    for i in tqdm(range(len(freq_steps[1:-1]))):
        f = freq_steps[i+1]
        popt, pcov = scipy.optimize.curve_fit(Lorentzian_power, xdata, ydata, sigma = sigma, p0 = [f, 2.0, 0.1, pl_popt[0], pl_popt[1]], \
                                              bounds=np.array([(f - (f-freq_steps[i])/2., f + (freq_steps[i+2] - f)/2.0), (1.0,np.inf), (0.0,np.inf), (0.0, np.inf), (-np.inf, 0.0)]).T)
        chisq.append(np.sum(((ydata - Lorentzian_power(xdata, *popt)) / sigma) ** 2))
    dof = len(xdata)-len(popt)
    return freq_steps[1:-1], chisq0, np.array(chisq), dof


def sim_Poisson_cospectra(lc_len, dt, mean_rate_A, mean_rate_B, n_lc):
    # Simulate a number of cross spectra due to Poisson noise.
    
    cross_spectra = []
    lc_times = np.linspace(0.,lc_len, int(np.floor(lc_len/dt)))
    
    for i in range(n_lc):
        sim_lc_counts_A = np.random.poisson(mean_rate_A*dt, size = np.shape(lc_times))
        sim_lc_counts_B = np.random.poisson(mean_rate_B*dt, size = np.shape(lc_times))
        lcA = lc.Lightcurve(lc_times, sim_lc_counts_A, dt=dt)
        lcB = lc.Lightcurve(lc_times, sim_lc_counts_B, dt=dt)
        cross = crossspec.Crossspectrum(lcA, lcB, norm='leahy')
        cross_spectra.append(cross)
    
    averaged_cross = cross_spectra[0]

    averaged_cross.m = len(cross_spectra)

    for i in range(len(cross_spectra))[1:]:
        averaged_cross.power += cross_spectra[i].power
        averaged_cross.unnorm_power += cross_spectra[i].unnorm_power
        averaged_cross.power_err += np.square(cross_spectra[i].power_err)
        averaged_cross.nphots1 += cross_spectra[i].nphots1
        averaged_cross.nphots2 += cross_spectra[i].nphots2

    averaged_cross.power = averaged_cross.power/averaged_cross.m
    averaged_cross.unnorm_power = averaged_cross.unnorm_power/averaged_cross.m
    averaged_cross.power_err = np.sqrt(averaged_cross.power_err)/averaged_cross.m
    averaged_cross.nphots1 = averaged_cross.nphots1/averaged_cross.m
    averaged_cross.nphots2 = averaged_cross.nphots2/averaged_cross.m
    
    return averaged_cross

def QPO_sim(Q, A, peakf, lc_len, dt, mean_rate_A, mean_rate_B, n_lc, f_min, f_max, num_trials = 100):
    # Simulate many cospectra due to Poisson noise then model with and without a Lorentzian QPO.
    # Return the distribution of resulting improvements to the fit.
    
    delta_chisq = []

    for i in tqdm(range(num_trials)):
        cross = sim_Poisson_cospectra(lc_len, dt, mean_rate_A, mean_rate_B, n_lc)
        f_mask = (cross.freq > f_min) * (cross.freq < f_max)
        xdata = cross.freq[f_mask]
        ydata = cross.power[f_mask]
        sigma = cross.power_err[f_mask]
        chisq0 = np.sum(((ydata -np.mean(ydata)) / sigma) ** 2)
        popt, pcov = scipy.optimize.curve_fit(Lorentzian_C, xdata, ydata, sigma = sigma, p0 = [peakf, Q, A, 0.0], \
                                              bounds =np.array([(f_min, f_max), (0.99*Q, 1.01*Q), (0.99*A, 1.01*A), (-np.inf, np.inf)]).T)
        chisq = np.sum(((ydata - Lorentzian_C(xdata, *popt)) / sigma) ** 2)
        delta_chisq.append(chisq - chisq0)
    
    return delta_chisq

def log_likelihood(theta, f, p, p_err):
    # ln(likelihood of data given model)
    log_f = theta[-1]
    # Model is 2 Lorentzians plus a constant
    model = N_Lorentzians(f, p, *(theta[:-1]))
    
    sigma2 = p_err ** 2 + model ** 2 * np.exp(2 * log_f)
    return -0.5 * np.sum((p - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta, *mu_sigma):
    # ln(likelihood of model). Gaussian priors given by results of least square fit.
    # mu_sigma gives the mean and std dev of gaussian prior for each peak frequency, Q, and A.
    # It has the form [*peak frequencies mu, *Q mu, *A mu, *peak frequencies sigma, *Q sigma, *A sigma]
    # theta should have form [*peak frequencies, *Q, *A, C, log_f]
    N = int((len(theta)-2)/3)
    log_f = theta[-1]
    prior_params = np.array(theta[:-2])
    
    mu = np.array(mu_sigma[:N*3])
    sigma = mu_sigma[N*3:]
#     print((N))
    
    if (not np.sum(prior_params <= 0.0)) and (-10.0 < log_f < 1.0):
        return -np.sum(np.square((prior_params-mu)/sigma))/2.0
    else:
        return -np.inf

def log_probability(theta, f, p, p_err, *mu_sigma):
    # ln(likelihood of model given data) also the posterior
    
    lp = log_prior(theta, *mu_sigma)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, f, p, p_err)

def fit_peaks(xdata, ydata, sigma, nu_peak):
    
    popt_arr = []
    pcov_arr = []

    for i, p in enumerate(nu_peak):
        f_bound = None
        if len(nu_peak)==1:
            f_bound = (np.min(xdata), np.max(xdata))
        else:
            if i == 0:
                f_bound = (np.min(xdata), p + (nu_peak[i+1] - p)/2)
            elif i==len(nu_peak)-1:
                f_bound = (p + (nu_peak[i-1] - p)/2, np.max(xdata))
            else:
                f_bound = (p + (nu_peak[i-1] - p)/2, p + (nu_peak[i+1] - p)/2)

        par_bounds = np.array([f_bound, (1.0,np.inf), (0, np.inf), (0, np.inf), (-np.inf, 0.0)]).T
        p0 = [p, 5.0, 0.1, 0.02, -0.5]
        popt, pcov = scipy.optimize.curve_fit(Lorentzian_power, xdata, ydata, sigma = sigma, p0 = p0, bounds = par_bounds)
        popt_arr.append(popt)
        pcov_arr.append(pcov)

    popt_arr = np.array(popt_arr)
    pcov_arr = np.array(pcov_arr)

    return popt_arr, pcov_arr




In [2]:
if platform=='linux' or platform=='linux2':
    print('Working on SRL server')
    root_dir = '/disk/lif2/spike/GRS_1741d9m2853/'
elif platform=='darwin':
    print('Working on Macbook')
    root_dir = '/Volumes/Samsung_1TB/AstroData/GRS_1741d9m2853/'

timing_dir = root_dir + 'timing_products/'
products_dir = root_dir + 'products/'
plot_dir = root_dir + 'figures/'

OBSID = '90601317002'

Working on Macbook


In [3]:
burst1_gti = list(fits.open(root_dir + 'burst1/' + 'burst1_gti.fits')[1].data)
burst2_gti = list(fits.open(root_dir + 'burst2/' + 'burst2_gti.fits')[1].data)

burst_gti = [*burst1_gti, *burst2_gti]

persistent_gti = list(fits.open(root_dir + 'persistent_gti.fits')[1].data)

pre_burst1_gti = [[burst1_gti[0][0] - 10000, burst1_gti[0][0]]]
pre_burst2_gti = [[burst2_gti[0][0] - 10000, burst2_gti[0][0]]]

print(burst1_gti)
print(burst2_gti)
print(burst_gti)
print(persistent_gti)
print(pre_burst1_gti)
print(pre_burst2_gti)

[(326544755.2406748, 326544795.7752123)]
[(326584617.41875273, 326584641.30960333)]
[(326544755.2406748, 326544795.7752123), (326584617.41875273, 326584641.30960333)]
[(326531992.76089, 326544655.2406748), (326545295.7752123, 326584517.41875273), (326585141.30960333, 326587440.76089)]
[[326534755.2406748, 326544755.2406748]]
[[326574617.41875273, 326584617.41875273]]


In [4]:
PI_min = 35     # 3.0 keV
# PI_min = 260     # 12.0 keV
PI_max = 960   # 40.0 keV
# PI_max = 1909   # 78.0 keV
events = extract_events(timing_dir + 'nu' + OBSID + 'A01_cl_barycorr.evt', \
            timing_dir + 'nu' + OBSID + 'B01_cl_barycorr.evt')

centroid_A = [535.45075,442.58818]
centroid_B = [529.34253,440.82937]
extraction_radius = 40.681706

events[0].set_xy_weights(centroid=centroid_A)
events[1].set_xy_weights(centroid=centroid_B)
# joined_events = events[0].join(events[1])
# print(events[0].time)

# plt.ion()

# curveA = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_sr.lc'), nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'A01_bk.lc'))
# curveB = bkg_subtract(nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_sr.lc'), nuproducts_to_stingray_lc(products_dir + 'nu' + OBSID + 'B01_bk.lc'))
# curve_total = sum_lc(curveA, curveB)
# # curve_10s = curve_total.rebin(dt_new=10)

# t_start = np.min(curve_total.time)

# plt.figure(figsize = (9,6))
# plt.errorbar(curve_total.time-t_start, curve_total.countrate, xerr=curve_total.dt/2., yerr=curve_total.countrate_err, fmt='none', lw = 0.5)
# plt.xlabel('Time (s)')
# plt.ylabel('NuSTAR count rate')
# plt.close()






/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:375: RuntimeWarning: underflow encountered in exp
  (c * ydiff ** 2)))
/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:399: RuntimeWarning: underflow encountered in exp
  (c * ydiff2)))
/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:409: RuntimeWarning: underflow encountered in true_divide
  dg_dA = g / amplitude
/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:410: RuntimeWarning: underflow encountered in multiply
  dg_dx_mean = g * ((2. * a * xdiff) + (b * ydiff))
/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:411: RuntimeWarning: underflow encountered in multiply
  dg_dy_mean = g * ((b * xdiff) + (2. * c * ydiff))
/Users/sean/anaconda/lib/python3.6/site-packages/astropy/modeling/functional_models.py:414: RuntimeWarning: underflow encountered i

array([2.11557975e-001, 1.33306525e-197, 1.67478870e-003, ...,
       7.00905624e-009, 4.81768486e-012, 2.68510653e-006])

## Persistent Cospectra

In [5]:
gti_lens = np.array([(g[1]-g[0]) for g in sting_gti.cross_two_gtis(events[0].gti, persistent_gti)])
len_mask = gti_lens > 500.
split_time = minimize_remainder(gti_lens[len_mask], 500,1500)
print(split_time)
ms_bin = 0.0001
f_res = 0.05

# print(events[0].split_by_time(bintime=split_time))
curves_A = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius) for x in events[0].split_by_time(bintime=split_time, gti=persistent_gti)]
# curves_B = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius) for x in events[1].split_by_time(bintime=split_time, gti=persistent_gti)]

n_curves = len(curves_A)

curves_A=[]



530.3030303030303


In [6]:
cross_spectra = []
for i in tqdm(range(n_curves)):
    cross_file = open(timing_dir + 'analysis_products/persistent_cross_spectrum_' + str(int(split_time)) + 's_segment' + str(i) + '.txt', 'rb')
    cross = pickle.load(cross_file)
    cross_file.close()
    cross_spectra.append(cross)


100%|██████████| 55/55 [01:18<00:00,  1.43s/it]


In [7]:
# averaged_cross = cross_spectra[0]

# averaged_cross.m = len(cross_spectra)

# for i in range(len(cross_spectra))[1:]:
#     averaged_cross.power += cross_spectra[i].power
#     averaged_cross.unnorm_power += cross_spectra[i].unnorm_power
#     averaged_cross.power_err += np.square(cross_spectra[i].power_err)
#     averaged_cross.nphots1 += cross_spectra[i].nphots1
#     averaged_cross.nphots2 += cross_spectra[i].nphots2
    
# averaged_cross.power = averaged_cross.power/averaged_cross.m
# averaged_cross.unnorm_power = averaged_cross.unnorm_power/averaged_cross.m
# averaged_cross.power_err = np.sqrt(averaged_cross.power_err)/averaged_cross.m
# averaged_cross.nphots1 = averaged_cross.nphots1/averaged_cross.m
# averaged_cross.nphots2 = averaged_cross.nphots2/averaged_cross.m

# cross_file = open(timing_dir + 'analysis_products/' + 'persistent_cross_spectrum_average.txt', 'wb')
# pickle.dump(averaged_cross, cross_file)
# cross_file.close()

cross_file = open(timing_dir + 'analysis_products/' + 'persistent_cross_spectrum_average.txt', 'rb')
averaged_cross = pickle.load(cross_file)
cross_file.close()


# Scan for QPOs

In [8]:
f_min = np.min(averaged_cross.freq)
f_max = np.min([2000., (np.max(averaged_cross.freq))])
freqs, chisq0, chisq, dof = QPO_scan(averaged_cross, f_min=f_min, f_max=f_max, f_bin = 500)

  0%|          | 0/500 [00:00<?, ?it/s]

[ 0.00344522 -1.11135956]


100%|██████████| 500/500 [34:52<00:00,  4.18s/it] 


In [9]:
print(chisq0/dof)
print(chisq0)
plt.figure(figsize=(9,6))
plt.plot(freqs, (chisq - chisq0))
plt.xscale('log')
plt.ylabel(r'$\Delta\chi^{2}$')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()
plt.savefig(plot_dir + 'persistent_QPO_scan.pdf')
plt.close()

0.9981585614537357
1058645.9721192706


In [17]:
peaks, _ = scipy.signal.find_peaks(-(chisq - chisq0), height = 8.0)
QPO_candidates = []
f_mask = (averaged_cross.freq > f_min) * (averaged_cross.freq < f_max) 
xdata = averaged_cross.freq[f_mask]
ydata = averaged_cross.power[f_mask]
sigma = averaged_cross.power_err[f_mask]

popt_arr, pcov_arr = fit_peaks(xdata, ydata, sigma, freqs[peaks])

sigma_arr = []

for i in range(len(popt_arr)):
    print(popt_arr[i])
#     print(np.sqrt(np.diag(pcov_arr[i])))
    sigma_arr.append(np.sqrt(np.diag(pcov_arr[i])))
sigma_arr = np.array(sigma_arr)

for i in range(len(popt_arr)):
    print(popt_arr[i][2]/sigma_arr[i][2])
    

[ 4.51408773e-01  3.21151101e+00  2.90147567e-03  3.25475242e-03
 -1.12223000e+00]
[ 7.99117821e+00  8.66153205e+00  2.87714580e-02  3.31034986e-03
 -1.11915258e+00]
[ 4.69729297e+01  6.13757239e+00  3.52055470e-02  3.41275890e-03
 -1.11321496e+00]
[ 1.16009183e+03  1.99472755e+01  6.76654644e-02  3.44201168e-03
 -1.11154439e+00]
0.34343995661903925
1.4443118130598473
0.6144929453429928
0.4289793305126663


In [30]:
plt.ion()

f_res = 0.05
plt.figure(figsize=(9,6))
averaged_cross_log = averaged_cross.rebin_log(f=f_res)
temp_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power.real, xerr=temp_err, yerr=averaged_cross_log.power_err, fmt='none', lw=0.5)
plt.plot(averaged_cross_log.freq, Lorentzian_power(averaged_cross_log.freq, *(popt_arr[0])), color='red')
plt.xscale('log')
plt.ylim((1e-4,1.))
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Leahy Power')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'persistent_averaged_cross_spectrum_' + str(int(split_time)) + 's_QPO_fit.pdf')
plt.close()



## Burst Cospectra

In [40]:
gti_lens = np.array([(g[1]-g[0]) for g in sting_gti.cross_two_gtis(events[0].gti, burst_gti)])
split_time = minimize_remainder(gti_lens, 10.,1000.)
print(split_time)
ms_bin = 0.0001
f_res = 0.05

# print(events[0].split_by_time(bintime=split_time))
curves_A = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius) for x in events[0].split_by_time(bintime=split_time, gti=burst_gti)]
curves_B = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius) for x in events[1].split_by_time(bintime=split_time, gti=burst_gti)]

print(curves_A[0].time)

10.0
[3.26544755e+08 3.26544755e+08 3.26544755e+08 ... 3.26544765e+08
 3.26544765e+08 3.26544765e+08]


In [41]:
cross_spectra = []
for i in tqdm(range(len(curves_A))):
    cross_file = open(timing_dir + 'analysis_products/burst_cross_spectrum_' + str(int(split_time)) + 's_segment' + str(i) + '.txt', 'rb')
    cross = pickle.load(cross_file)
    cross_file.close()
    cross_spectra.append(cross)


100%|██████████| 6/6 [00:00<00:00, 31.54it/s]


In [42]:
plt.ion()

averaged_cross = cross_spectra[0]

averaged_cross.m = len(cross_spectra)

for i in range(len(cross_spectra))[1:]:
    averaged_cross.power += cross_spectra[i].power
    averaged_cross.unnorm_power += cross_spectra[i].unnorm_power
    averaged_cross.power_err += np.square(cross_spectra[i].power_err)
    averaged_cross.nphots1 += cross_spectra[i].nphots1
    averaged_cross.nphots2 += cross_spectra[i].nphots2
    
averaged_cross.power = averaged_cross.power/averaged_cross.m
averaged_cross.unnorm_power = averaged_cross.unnorm_power/averaged_cross.m
averaged_cross.power_err = np.sqrt(averaged_cross.power_err)/averaged_cross.m
averaged_cross.nphots1 = averaged_cross.nphots1/averaged_cross.m
averaged_cross.nphots2 = averaged_cross.nphots2/averaged_cross.m


In [52]:
tmp_mask = (averaged_cross.freq <= 590) * (averaged_cross.freq >= 588)

# rms = crossspec.normalize_crossspectrum(averaged_cross.unnorm_power, 10, )
print(np.sum(averaged_cross.power[tmp_mask])*10/np.sqrt(np.exp(np.log(averaged_cross.nphots1) + np.log(averaged_cross.nphots2))))
print(np.sqrt(np.sum(np.square(averaged_cross.power_err[tmp_mask])))*2.6*10/np.sqrt(np.exp(np.log(averaged_cross.nphots1) + np.log(averaged_cross.nphots2))))

-0.024428886330163685
0.09870024807360071


# Scan for QPOs

In [21]:
f_min = np.min(averaged_cross.freq)
f_max = np.min([2000., (np.max(averaged_cross.freq))])
freqs, chisq0, chisq, dof = QPO_scan(averaged_cross, f_min=f_min, f_max=f_max, f_bin = 500)

  0%|          | 2/500 [00:00<00:29, 17.02it/s]

[ 0.26232159 -2.12672135]


100%|██████████| 500/500 [00:46<00:00, 10.83it/s]


In [22]:
print(chisq0/dof)
plt.figure(figsize=(9,6))
plt.plot(freqs, (chisq - chisq0))
plt.xscale('log')
plt.ylabel(r'$\Delta\chi^{2}$')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()
plt.savefig(plot_dir + 'burst_QPO_scan.pdf')
plt.close()

1.0192679782822567


In [24]:
peaks, _ = scipy.signal.find_peaks(-(chisq - chisq0), height = 10.0)
QPO_candidates = []
f_mask = (averaged_cross.freq > f_min) * (averaged_cross.freq < f_max) 
xdata = averaged_cross.freq[f_mask]
ydata = averaged_cross.power[f_mask]
sigma = averaged_cross.power_err[f_mask]

popt_arr, pcov_arr = fit_peaks(xdata, ydata, sigma, freqs[peaks])

sigma_arr = []

for i in range(len(popt_arr)):
    print(popt_arr[i])
    print(np.sqrt(np.diag(pcov_arr[i])))
    sigma_arr.append(np.sqrt(np.diag(pcov_arr[i])))
sigma_arr = np.array(sigma_arr)

for i in range(len(popt_arr)):
    print(popt_arr[i][2]/sigma_arr[i][2])

    

[ 1.18703532e+03  1.29631149e+02  2.21004771e+00  2.62263425e-01
 -2.12687100e+00]
[ 1.85097360e+03  1.61683559e+02  3.94975355e+00  2.62263748e-01
 -2.12687020e+00]
1.6209724425999592
2.5913252335692993


In [25]:
plt.ion()

f_res = 0.05
plt.figure(figsize=(9,6))
averaged_cross_log = averaged_cross.rebin_log(f=f_res)
temp_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power.real, xerr=temp_err, yerr=averaged_cross_log.power_err, fmt='none', lw=0.5)
plt.plot(averaged_cross_log.freq, Lorentzian_power(averaged_cross_log.freq, *(popt_arr[1])), color='red')
plt.xscale('log')
plt.ylim((1e-6,100.))
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Leahy Power')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'burst_averaged_cross_spectrum_' + str(int(split_time)) + 's_QPO_fit.pdf')
plt.close()



## Pre-Burst 1 Cospectra

In [26]:
gti_lens = np.array([(g[1]-g[0]) for g in sting_gti.cross_two_gtis(events[0].gti, pre_burst1_gti)])
split_time = minimize_remainder(gti_lens, 500.,2000.)
print(split_time)
ms_bin = 0.0001
f_res = 0.05

# print(events[0].split_by_time(bintime=split_time))
curves_A = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius) for x in events[0].split_by_time(bintime=split_time, gti=pre_burst1_gti)]
curves_B = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius) for x in events[1].split_by_time(bintime=split_time, gti=pre_burst1_gti)]

print(curves_A[0].time)

515.1515151515151
[3.26534755e+08 3.26534755e+08 3.26534755e+08 ... 3.26535270e+08
 3.26535270e+08 3.26535270e+08]


In [27]:
cross_spectra = []
for i in tqdm(range(len(curves_A))):
    cross_file = open(timing_dir + 'analysis_products/preburst1_cross_spectrum_' + str(int(split_time)) + 's_segment' + str(i) + '.txt', 'rb')
    cross = pickle.load(cross_file)
    cross_file.close()
    cross_spectra.append(cross)


100%|██████████| 9/9 [00:12<00:00,  1.42s/it]


In [28]:
plt.ion()

averaged_cross = cross_spectra[0]

averaged_cross.m = len(cross_spectra)

for i in range(len(cross_spectra))[1:]:
    averaged_cross.power += cross_spectra[i].power
    averaged_cross.unnorm_power += cross_spectra[i].unnorm_power
    averaged_cross.power_err += np.square(cross_spectra[i].power_err)
    averaged_cross.nphots1 += cross_spectra[i].nphots1
    averaged_cross.nphots2 += cross_spectra[i].nphots2
    
averaged_cross.power = averaged_cross.power/averaged_cross.m
averaged_cross.unnorm_power = averaged_cross.unnorm_power/averaged_cross.m
averaged_cross.power_err = np.sqrt(averaged_cross.power_err)/averaged_cross.m
averaged_cross.nphots1 = averaged_cross.nphots1/averaged_cross.m
averaged_cross.nphots2 = averaged_cross.nphots2/averaged_cross.m


# Scan for QPOs

In [29]:
f_min = np.min(averaged_cross.freq)
f_max = np.min([2000., (np.max(averaged_cross.freq))])
freqs, chisq0, chisq, dof = QPO_scan(averaged_cross, f_min=f_min, f_max=f_max, f_bin = 500)

  0%|          | 0/500 [00:00<?, ?it/s]

[ 6.09805232e-08 -1.60319817e-01]


 79%|███████▉  | 397/500 [27:50<06:59,  4.07s/it]/Users/sean/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: underflow encountered in power
/Users/sean/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: underflow encountered in multiply
100%|██████████| 500/500 [32:25<00:00,  3.89s/it]


In [30]:
print(chisq0/dof)
plt.figure(figsize=(9,6))
plt.plot(freqs, (chisq - chisq0))
plt.xscale('log')
plt.ylabel(r'$\Delta\chi^{2}$')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()
plt.savefig(plot_dir + 'preburst1_QPO_scan.pdf')
plt.close()

1.027641820648177


In [32]:
peaks, _ = scipy.signal.find_peaks(-(chisq - chisq0), height = 8.0)
QPO_candidates = []
f_mask = (averaged_cross.freq > f_min) * (averaged_cross.freq < f_max) 
xdata = averaged_cross.freq[f_mask]
ydata = averaged_cross.power[f_mask]
sigma = averaged_cross.power_err[f_mask]

popt_arr, pcov_arr = fit_peaks(xdata, ydata, sigma, freqs[peaks])

sigma_arr = []

for i in range(len(popt_arr)):
    print(popt_arr[i])
    print(np.sqrt(np.diag(pcov_arr[i])))
    sigma_arr.append(np.sqrt(np.diag(pcov_arr[i])))
sigma_arr = np.array(sigma_arr)

for i in range(len(popt_arr)):
    print(popt_arr[i][2]/sigma_arr[i][2])

    

[ 1.44711285e+01  6.37231953e+00  4.99375301e-02  6.51663862e-07
 -6.56217282e-01]
[1.77887421e+00 1.45579839e+01 8.28404272e-02 7.13247824e-03
 2.77195424e+03]
0.6028159417161971


In [39]:
plt.ion()

f_res = 0.05
plt.figure(figsize=(9,6))
averaged_cross_log = averaged_cross.rebin_log(f=f_res)
temp_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power.real, xerr=temp_err, yerr=averaged_cross_log.power_err, fmt='none', lw=0.5)
plt.plot(averaged_cross_log.freq, Lorentzian_power(averaged_cross_log.freq, *(popt_arr[1])), color='red')
plt.xscale('log')
plt.ylim((1e-6,100.))
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Leahy Power')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'preburst1_averaged_cross_spectrum_' + str(int(split_time)) + 's_QPO_fit.pdf')
plt.close()



## Pre-Burst 2 Cospectra

In [33]:
gti_lens = np.array([(g[1]-g[0]) for g in sting_gti.cross_two_gtis(events[0].gti, pre_burst2_gti)])
split_time = minimize_remainder(gti_lens, 500.,2000.)
print(split_time)
ms_bin = 0.0001
f_res = 0.05

# print(events[0].split_by_time(bintime=split_time))
curves_A = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_A, radius = extraction_radius) for x in events[0].split_by_time(bintime=split_time, gti=pre_burst2_gti)]
curves_B = [x.to_lc(dt = ms_bin, pi_low=PI_min, pi_high=PI_max, centroid = centroid_B, radius = extraction_radius) for x in events[1].split_by_time(bintime=split_time, gti=pre_burst2_gti)]

print(curves_A[0].time)

545.4545454545455
[3.26574617e+08 3.26574617e+08 3.26574617e+08 ... 3.26575163e+08
 3.26575163e+08 3.26575163e+08]


In [34]:
cross_spectra = []
for i in tqdm(range(len(curves_A))):
    cross_file = open(timing_dir + 'analysis_products/preburst2_cross_spectrum_' + str(int(split_time)) + 's_segment' + str(i) + '.txt', 'rb')
    cross = pickle.load(cross_file)
    cross_file.close()
    cross_spectra.append(cross)


100%|██████████| 8/8 [00:11<00:00,  1.49s/it]


In [35]:
plt.ion()

averaged_cross = cross_spectra[0]

averaged_cross.m = len(cross_spectra)

for i in range(len(cross_spectra))[1:]:
    averaged_cross.power += cross_spectra[i].power
    averaged_cross.unnorm_power += cross_spectra[i].unnorm_power
    averaged_cross.power_err += np.square(cross_spectra[i].power_err)
    averaged_cross.nphots1 += cross_spectra[i].nphots1
    averaged_cross.nphots2 += cross_spectra[i].nphots2
    
averaged_cross.power = averaged_cross.power/averaged_cross.m
averaged_cross.unnorm_power = averaged_cross.unnorm_power/averaged_cross.m
averaged_cross.power_err = np.sqrt(averaged_cross.power_err)/averaged_cross.m
averaged_cross.nphots1 = averaged_cross.nphots1/averaged_cross.m
averaged_cross.nphots2 = averaged_cross.nphots2/averaged_cross.m


# Scan for QPOs

In [36]:
f_min = np.min(averaged_cross.freq)
f_max = np.min([2000., (np.max(averaged_cross.freq))])
freqs, chisq0, chisq, dof = QPO_scan(averaged_cross, f_min=f_min, f_max=f_max, f_bin = 500)

  0%|          | 0/500 [00:00<?, ?it/s]

[ 0.00573029 -1.14601813]


100%|██████████| 500/500 [41:32<00:00,  4.99s/it]   


In [37]:
print(chisq0/dof)
plt.figure(figsize=(9,6))
plt.plot(freqs, (chisq - chisq0))
plt.xscale('log')
plt.ylabel(r'$\Delta\chi^{2}$')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()
plt.savefig(plot_dir + 'preburst2_QPO_scan.pdf')
plt.close()

1.030140714818998


In [38]:
peaks, _ = scipy.signal.find_peaks(-(chisq - chisq0), height = 10.0)
QPO_candidates = []
f_mask = (averaged_cross.freq > f_min) * (averaged_cross.freq < f_max) 
xdata = averaged_cross.freq[f_mask]
ydata = averaged_cross.power[f_mask]
sigma = averaged_cross.power_err[f_mask]

popt_arr, pcov_arr = fit_peaks(xdata, ydata, sigma, freqs[peaks])

sigma_arr = []

for i in range(len(popt_arr)):
    print(popt_arr[i])
    print(np.sqrt(np.diag(pcov_arr[i])))
    sigma_arr.append(np.sqrt(np.diag(pcov_arr[i])))
sigma_arr = np.array(sigma_arr)

for i in range(len(popt_arr)):
    print(popt_arr[i][2]/sigma_arr[i][2])

    

[ 1.73733392e-02  1.35002956e+02  5.50849671e-02  1.42817803e-03
 -1.40090608e+00]
[1.74857166e-04 3.93590009e+04 1.58998664e+01 1.81160661e-03
 2.35515970e-01]
[ 7.87857342e+01  7.18209095e+00  1.31618821e-01  5.61321094e-03
 -1.15009153e+00]
[7.37865805e+00 1.36870264e+01 1.77158553e-01 3.68243051e-03
 1.27989412e-01]
[ 3.09919389e+02  8.45994250e+01  2.61290439e-01  5.65406621e-03
 -1.14868227e+00]
[7.17291272e-01 4.68523295e+01 1.02322029e-01 3.69850717e-03
 1.27652857e-01]
0.003464492443763883
0.7429436432019411
2.553608855575088


In [39]:
plt.ion()

f_res = 0.05
plt.figure(figsize=(9,6))
averaged_cross_log = averaged_cross.rebin_log(f=f_res)
temp_err = averaged_cross.df*np.power(1.+f_res, range(len(averaged_cross_log.freq)))/2.
plt.errorbar(averaged_cross_log.freq, averaged_cross_log.power.real, xerr=temp_err, yerr=averaged_cross_log.power_err, fmt='none', lw=0.5)
plt.plot(averaged_cross_log.freq, Lorentzian_power(averaged_cross_log.freq, *(popt_arr[2])), color='red')
plt.xscale('log')
plt.ylim((1e-6,100.))
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Leahy Power')
plt.tight_layout()
# plt.show()
plt.savefig(plot_dir + 'preburst2_averaged_cross_spectrum_' + str(int(split_time)) + 's_QPO_fit.pdf')
plt.close()

